# Setup

In [1]:
import pandas as pd

# Loading the Data

In [2]:
p1 = pd.read_parquet("data/X_train.part1.parquet")
p2 = pd.read_parquet("data/X_train.part2.parquet")
p3 = pd.read_parquet("data/X_train.part3.parquet")
X_train = pd.concat([p1, p2, p3], axis=0)
X_train.to_parquet("data/X_train.parquet", index=True)

In [3]:
X_train = pd.read_parquet("data/X_train.parquet")
X_test = pd.read_parquet("data/X_test.reduced.parquet")
X_test = [g for _, g in X_test.groupby(level="id", sort=False)]
y_train = pd.read_parquet("data/y_train.parquet")["structural_breakpoint"]
y_test = pd.read_parquet("data/y_test.reduced.parquet")["structural_breakpoint"]
print(len(y_test))

101


In [4]:
# Import holdout ids
ids_holdout = pd.read_csv("data/ids_holdout.csv")["id"]
print(len(ids_holdout))

# Select select rows belonging to those ids
X_holdout = X_train.loc[X_train.index.get_level_values("id").isin(ids_holdout)]
y_holdout = y_train.loc[y_train.index.isin(ids_holdout)]

# Now select the train set
X_train = X_train.loc[~X_train.index.get_level_values("id").isin(ids_holdout)]
y_train = y_train.loc[~y_train.index.isin(ids_holdout)]

401


In [5]:
# Merge X_test with X_holdout
X_holdout_list = [g for _, g in X_holdout.groupby(level="id")]
X_test = X_test + X_holdout_list
y_test = pd.concat([y_test, y_holdout])
print(len(y_test))

502


# Feature Extraction

In [6]:
from src.feature_extraction import build_features

X_train_feat = build_features(X_train)

In [7]:
from src.preprocess import _latest_cache

path = _latest_cache("preprocessed")
X_preprocess = pd.read_parquet(path)

In [8]:
X_preprocess

original  period  standardized   clipped  detrended  \
id    time                                                        
0     0    -0.005564       0     -0.815097 -0.815097  -0.782390   
      1     0.003705       0      0.493405  0.493405   0.526106   
      2     0.013164       0      1.828686  1.828686   1.861380   
      3     0.007151       0      0.979816  0.979816   1.012503   
      4    -0.009979       0     -1.438248 -1.438248  -1.405568   
...              ...     ...           ...       ...        ...   
10000 2134  0.001137       1      0.172125  0.172125   0.182797   
      2135  0.003526       1      0.534042  0.534042   0.544894   
      2136  0.000687       1      0.103832  0.103832   0.114863   
      2137  0.001640       1      0.248225  0.248225   0.259437   
      2138  0.001074       1      0.162523  0.162523   0.173914   

            diff_standardized  diff_detrended  absdiff_detrended  \
id    time                                                         
0     0              0.008731       -0.000417          -0.778978   
      1              0.896048        0.886899           0.108311   
      2              0.914207        0.905057           0.126442   
      3             -0.566901       -0.576051          -0.220891   
      4             -1.630998       -1.640148           0.843178   
...                       ...             ...                ...   
10000 2134           0.023905        0.012937          -0.388254   
      2135           0.256742        0.245771          -0.155537   
      2136          -0.283044       -0.294018          -0.129354   
      2137           0.108513        0.097536          -0.304005   
      2138          -0.048283       -0.059263          -0.364355   

            absval_detrended  squared_detrended  diff2_standardized  \
id    time                                                            
0     0             0.072128          -0.260540            0.017401   
      1            -0.472533          -0.681539            0.517190   
      2             1.788043           2.419041            0.027629   
      3             0.350876           0.034923           -0.816848   
      4             1.126955           1.143375           -0.581962   
...                      ...                ...                 ...   
10000 2134         -0.404325          -0.276297            0.227602   
      2135          0.247186          -0.020654            0.114081   
      2136         -0.527275          -0.295005           -0.321840   
      2137         -0.267346          -0.244101            0.203633   
      2138         -0.421630          -0.279233           -0.105754   

            diff2_detrended  
id    time                   
0     0           -0.001263  
      1            0.498527  
      2            0.008966  
      3           -0.835510  
      4           -0.600624  
...                     ...  
10000 2134         0.245193  
      2135         0.131673  
      2136        -0.304247  
      2137         0.221227  
      2138        -0.088158  

[22774288 rows x 12 columns]

In [9]:
X_train_feat

,scaled_med_orig_b,med_delta,mad_orig_b,mad_logratio,mean_vs_med,std_vs_mad,skew_contrast,kurt_contrast,slope_delta,Q5_delta,...,roll_logstd_jump_w200_topkabs_mean_delta,roll_rms_jump_w200_maxpos_delta,roll_rms_jump_w200_maxneg_delta,roll_rms_jump_w200_topkabs_mean_delta,roll_crossrate_logit_w200_maxpos_delta,roll_crossrate_logit_w200_maxneg_delta,roll_crossrate_logit_w200_topkabs_mean_delta,ewvar_hl200_logratio,ewvar_hl400_logratio,ar_ridge_nll_logratio
id,,,,,,,,,,,,,,,,,,,,,
0,0.043904,-0.030569,-5.343694,0.007507,0.029409,-0.024808,0.046588,-0.376850,0.000406,-0.047972,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.177336,1.613605,-0.025269
1,0.096387,-0.059507,-6.664820,-0.115805,-0.056037,-0.100933,-1.021112,1.679963,0.000260,0.318995,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.143927,-0.419240,-0.024284
2,0.071892,0.046430,-4.665885,0.257437,0.053943,0.026775,0.421911,1.663851,-0.000031,-0.493941,...,-0.424092,-0.130476,1.333052,-0.359579,-0.158753,0.970515,-0.602217,0.730782,0.212561,0.112243
3,0.056101,-0.036831,-5.331440,0.087346,0.029188,0.013754,0.200757,-0.566938,-0.000110,-0.142318,...,-0.199354,-0.132738,0.556890,-0.134405,-0.021406,-0.057538,-0.021406,0.184961,-0.336105,0.033925
4,0.009689,0.031867,-6.092796,0.024234,-0.020023,0.003079,0.296994,0.029632,-0.001864,-0.135689,...,-0.100130,-0.137243,0.136116,-0.142779,0.070774,0.490873,0.083676,-0.161101,-0.367566,-0.023485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,-0.350639,0.119104,-5.717372,-0.614162,-0.162056,0.162098,2.104231,72.557587,0.000005,0.790051,...,-0.084441,0.025205,0.521442,0.018366,-0.630383,0.601437,-0.625396,-0.635892,-1.226713,-0.112085
9997,0.045326,-0.017855,-5.527782,0.096171,0.081812,-0.088640,0.048792,-0.189005,-0.000526,0.227500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.734093,1.142978,0.040488
9998,0.003709,0.006437,-5.314662,0.011088,0.000672,-0.006384,0.122828,-0.040949,0.000009,-0.016283,...,-0.051295,-0.113934,0.084398,-0.101113,-0.095522,0.196294,-0.189617,0.039246,0.135868,-0.073071


# The `train()` Function

In [10]:
from src.stacking import train

# results = train_model(X_train_feat, y_train, model_type='xgb_main')
meta_results = train(X_train, y_train)

Loading cached data from resources/features/all_0122_1729.parquet

════════════════════════════════════════════════════════════════════════
═══════════════════════ TRAINING BASE — xgb_main ═══════════════════════
════════════════════════════════════════════════════════════════════════

────────────────────────────────────────────────────────────────────────
[xgb_main] SEED 169 (1/3) — START
[xgb_main] Fold 1/5: TRAIN 0.7169 | VAL 0.6672
[xgb_main] Fold 2/5: TRAIN 0.7068 | VAL 0.6826
[xgb_main] Fold 3/5: TRAIN 0.7262 | VAL 0.6832
[xgb_main] Fold 4/5: TRAIN 0.7347 | VAL 0.6459
[xgb_main] Fold 5/5: TRAIN 0.6842 | VAL 0.6873
[xgb_main] OOF AUC = 0.6598
[xgb_main] Full-data TRAIN AUC = 0.7251

────────────────────────────────────────────────────────────────────────
[xgb_main] SEED 186 (2/3) — START
[xgb_main] Fold 1/5: TRAIN 0.7067 | VAL 0.6774
[xgb_main] Fold 2/5: TRAIN 0.7232 | VAL 0.6733
[xgb_main] Fold 3/5: TRAIN 0.7257 | VAL 0.6895
[xgb_main] Fold 4/5: TRAIN 0.7264 | VAL 0.6494
[xgb_ma

# The `infer()` Function

# Local testing

To make sure your `train()` and `infer()` function are working properly, you can call the `crunch.test()` function that will reproduce the cloud environment locally. <br />
Even if it is not perfect, it should give you a quick idea if your model is working properly.

In [9]:
import crunch

crunch = crunch.load_notebook()

loaded inline runner with module: <module '__main__'>

cli version: 10.8.1
available ram: 16.00 gb
available cpu: 10 core
----


In [14]:
# You need to setup the notebook first, cf:
#  - https://github.com/crunchdao/competitions/blob/master/competitions/structural-break/quickstarters/baseline/baseline.ipynb
#  - https://hub.crunchdao.com/competitions/structural-break/submit
!crunch setup --notebook structural-break hello --token aaaabbbbccccddddeeeeffff

crunch-cli, version 10.8.1

---
Your token seems to have expired or is invalid.

Please follow this link to copy and paste your new setup command:
https://hub.crunchdao.com/competitions/structural-break/submit

If you think that is an error, please contact an administrator.


In [15]:
# crunch.test(
#     # Uncomment to disable the train
#     force_first_train=False,

#     # Uncomment to disable the determinism check
#     # no_determinism_check=True,
# )

# Results

Once the local tester is done, you can preview the result stored in `data/prediction.parquet`.

In [16]:
# prediction = pd.read_parquet("data/prediction.parquet")
# prediction

### Local scoring

You can call the function that the system uses to estimate your score locally.

In [17]:
# # Call the scoring function
# from sklearn.metrics import roc_auc_score
# roc_auc_score(
#     y_test,
#     prediction,
# )

# Submit your Notebook

To submit your work, you must:
1. Download your Notebook from Colab
2. Upload it to the platform
3. Create a run to validate it

### >> https://hub.crunchdao.com/competitions/structural-break/submit/notebook